# Example Budget

This is an example of how to use a Jupyter notebook to track your budget and
forecast your account balance. You can run it the same way you run any other
Jupyter notebook, but this example is best viewed in Visual Studio Code with the
Python, Jupyter, and Data Wrangler extensions installed (see
[`.vscode/extensions.json`][ext]).

[ext]: ./.vscode/extensions.json

Start by creating an account. There's nothing in it yet, so its balance is, of
course, zero:

In [7]:
from budge import Account

account = Account("Example account")
account.balance()

<stockholm.Money: "0.00">

Let's pretend that it's January 10, 2022, and we started using this notebook on
January 1, so the first transaction is our initial balance as of New Year's Eve:

In [8]:
from datetime import date

from stockholm import Money
from budge import Transaction

start_date = date(2021, 12, 31)
today = date(2022, 1, 10)

account.transactions.add(Transaction("Starting balance", Money(1500), start_date))
account.balance(start_date)

<stockholm.Money: "1500.00">

You can also import from local modules, just like with any other Jupyter
notebook. Here, `load_transactions` and `load_bills` in [`example.py`][ex] read
data from the given CSV files, parse them into `Transaction` or
`RepeatingTransaction` objects, and load them into the account.  The account's
balance as of today is now calculated using the loaded transactions and bills:

[ex]: ./example.py

In [9]:
from example import load_transactions, load_bills

load_transactions(account, "data/transactions.csv")
load_bills(account, "data/bills.csv")

account.balance(today)

<stockholm.Money: "263.41">

Now let's look at the transactions and bills that have come out of the account
so far. The `account.running_balance` method returns an iterator of transactions
in the account, and the current account balance after that transaction. We can
use that to create a Pandas dataframe and display it as a table for readability:

In [10]:
import pandas as pd

transactions = pd.DataFrame(
    (
        (transaction.date, transaction.amount, transaction.description, balance)
        for transaction, balance in account.running_balance(start_date, today)
    ),
    columns=["Date", "Amount", "Description", "Balance"],
)

transactions

,Date,Amount,Description,Balance
0,2021-12-31,1500.00,Starting balance,1500.00
1,2022-01-01,-800.00,Rent,700.00
2,2022-01-03,-50.00,Extra groceries,650.00
3,2022-01-04,-60.00,Internet,590.00
4,2022-01-06,-51.59,Gas,538.41
5,2022-01-07,-80.00,Groceries,458.41
6,2022-01-07,-45.00,Going out,413.41
7,2022-01-09,-150.00,Electric,263.41


Since we know of some transactions and bills that haven't happened yet, we can
pass future dates to `account.running_balance` to see what our account balance
will look like after each future transaction:

In [11]:
from dateutil.relativedelta import relativedelta

end_date = start_date + relativedelta(months=1)

future_transactions = pd.DataFrame(
    (
        (transaction.date, transaction.amount, transaction.description, balance)
        for transaction, balance in account.running_balance(
            today + relativedelta(days=1), end_date
        )
    )
)

future_transactions

,0,1,2,3
0,2022-01-12,-60.00,Cell phone,203.41
1,2022-01-13,-51.59,Gas,151.82
2,2022-01-14,1000.00,Paycheck,1151.82
3,2022-01-14,-80.00,Groceries,1071.82
4,2022-01-18,-100.00,Birthday gift,971.82
5,2022-01-20,-51.59,Gas,920.23
6,2022-01-21,-80.00,Groceries,840.23
7,2022-01-27,-51.59,Gas,788.64
8,2022-01-28,-80.00,Groceries,708.64
9,2022-01-28,1000.00,Paycheck,1708.64


And we can look at what our daily ending balance will be for any range of dates:

In [12]:
balances = pd.DataFrame(account.daily_balance(start_date, end_date))

balances

,date,balance
0,2021-12-31,1500.00
1,2022-01-01,700.00
2,2022-01-02,700.00
3,2022-01-03,650.00
4,2022-01-04,590.00
5,2022-01-05,590.00
6,2022-01-06,538.41
7,2022-01-07,413.41
8,2022-01-08,413.41
9,2022-01-09,263.41


And if you want to know what your account balance will be in a year, if none of
your bills change:

In [13]:
account.balance(today + relativedelta(years=1))

<stockholm.Money: "6480.73">